In [1]:
import random
from langchain_groq import ChatGroq

array = ['chicken','watermelom']

# Initial button state
buttonisgenerate = 1

llm = ChatGroq(
    temperature=0.7,  # Increased temperature for more variability
    groq_api_key=api_key,
    model_name="llama-3.1-70b-versatile"
)

# Store previously generated recipes
previous_recipes = set()

def generate_recipe(ingredients, button_state):
    # Add some variability to the prompt based on the button state
    variability = random.choice(["Try making", "How about a recipe for", "You can create"])
    prompt = f"{variability} a recipe using the following ingredients: {', '.join(ingredients)}. Just give me the title, ingredients, and instructions and nothing else.Also thell whether the generated recipe will be tasty(eatable)?"

    # Generate a recipe
    response = llm.invoke(prompt)
    return response.content

# Generate recipe based on button state
if buttonisgenerate == 1:
    response = generate_recipe(array, buttonisgenerate)
    previous_recipes.add(response)  # Store the generated recipe
    print(response)
    buttonisgenerate = 2  # Change state for the next button press
else:
    while True:
        response = generate_recipe(array, buttonisgenerate)
        if response not in previous_recipes:  # Ensure it's a new recipe
            previous_recipes.add(response)  # Store the new recipe
            print(response)
            break  # Exit the loop after generating a new recipe


NameError: name 'api_key' is not defined

In [2]:
import chromadb
import os

# Create a persistent ChromaDB client
client = chromadb.PersistentClient('food_items_vectorstore')

# Create or get the collection
collection = client.get_or_create_collection(name="food_items")

# Sample data to insert into ChromaDB
food_data = [
    {"name": "Milk", "expiry": "7 days"},
    {"name": "Bread", "expiry": "3 days"},
    {"name": "Butter", "expiry": "2 weeks"},
    {"name": "Eggs", "expiry": "4 weeks"},
    {"name": "Rice", "expiry": "6 months"}
]

# Check if the collection is empty, then add data
if not collection.count():
    for item in food_data:
        collection.add(
            documents=[item['name']],  # Use the name of the food item as the document
            metadatas={"expiry": item['expiry']},  # Store expiry information in metadata
            ids=[item['name']]  # Use food item name as the unique identifier
        )

print("Food items added to ChromaDB successfully!")



Food items added to ChromaDB successfully!


In [3]:
# Example query to find a food item by name
query_name = "Milk"  # Replace with any food item name to query
result = collection.query(
    query_texts=[query_name],
    n_results=1  # Number of results to return
)

print("Query Result:", result)


Query Result: {'ids': [['Milk']], 'distances': [[0.0]], 'metadatas': [[{'expiry': '7 days'}]], 'embeddings': None, 'documents': [['Milk']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [5]:
from datetime import datetime, timedelta
expiring_items = []
items = collection.get()  # Fetch the items from the collection
now = datetime.now()
for item in items['metadatas']:  # Access the correct key for metadata
        expiry = item['expiry']

        # Convert the expiry to an appropriate timedelta
        days_to_expire = 0
        if 'days' in expiry:
            days_to_expire = int(expiry.split()[0])
        elif 'weeks' in expiry:
            days_to_expire = int(expiry.split()[0]) * 7
        elif 'months' in expiry:
            days_to_expire = int(expiry.split()[0]) * 30
        
        expiry_date = now + timedelta(days=days_to_expire)

        # Check if the item is expiring within the next 3 days
        if now <= expiry_date <= now + timedelta(days=3):
            expiring_items.append(item)

print("Expiring Items:", expiring_items)

Expiring Items: [{'expiry': '3 days'}]
